In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pickle
import random
random.seed(42)
np.random.seed(42)

In [3]:
with open("../scripts/llm_dpo_rl_evaluated.pickle", "rb") as file:
#with open("../ressources/llm_dpo_rl_evaluated_25-01-20_15x100_on_s14_n300_24-12-17.pickle", "rb") as file:
#with open("../ressources/dpo_rl_01_24-12-24_s17_n600_e10_r100/dataset.pickle", "rb") as file:
#with open("../ressources/automode_evaluated_concat_s14s15s16_n300_24-12-18.pickle", "rb") as file:
#with open("../ressources/llm_evaluated_concat_s14n600_s15n600_train_2024-12-12_16.pickle", "rb") as file:
#with open("testdata.pkl", "rb") as file:
    original_df = pickle.load(file)

In [4]:
original_df

,level_0,index,description,configuration,parameters,argos,behavior_tree,scores,avg_score,type,lightstype,arenatype,robotstype,llm_scores,llm_behavior_tree,llm_avg_score,llm_dpo_rl_scores,llm_dpo_rl_behavior_tree,llm_dpo_rl_avg_score
0,0,0,The environment is a circular arena with radiu...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 4 --n0 0 --nchild0 2 --...,"[4.0, 2.0, 4.0, 5.0, 3.0, 4.0, 1.0, 3.0, 5.0, ...",3.600000,Foraging,TargetLights,CircularArena,CenteredSwarm,"[1.0, 2.0, 0.0, 1.0, 4.0, 0.0, 2.0, 2.0, 3.0, ...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,1.900000,"[4.0, 7.0, 3.0, 3.0, 4.0, 2.0, 2.0, 1.0, 5.0, ...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,3.400000
1,1,1,The environment consists of a circular arena w...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 1 --n0 0 --nchild0 2 --...,"[3.0, 3.0, 4.0, 2.0, 1.0, 2.0, 5.0, 3.0, 4.0, ...",2.900000,Foraging,UniformLights,CircularArena,CenteredSwarm,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,0.100000,"[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,0.200000
2,2,2,"With a radius of 3.44 meters, the circular are...","<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 --...,"[-8.57746, -4.79552, -7.16427, -9.41532, -6.30...",-7.018138,Connection,UniformLights,CircularArena,CenteredSwarm,"[-5.78745, -6.22081, -5.69374, -5.86581, -6.13...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,-6.210486,"[-1.30253, -1.22782, -1.30399, -1.00202, -1.85...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,-1.451391
3,3,3,The arena has a radius of 1.99 m. In this sett...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 --...,"[0.8, 0.866667, 0.733333, 0.8, 0.866667, 0.6, ...",0.740000,Aggregation,UniformLights,CircularArena,CenteredSwarm,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,0.000000
4,4,4,The area is a rectangle with dimensions 5.11 x...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 --...,"[3.0, 5.0, 6.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, ...",4.700000,Foraging,TargetLights,RectangularArena,CenteredSwarm,"[4.0, 2.0, 5.0, 4.0, 7.0, 3.0, 5.0, 3.0, 3.0, ...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,4.000000,"[4.0, 2.0, 0.0, 1.0, 3.0, 3.0, 2.0, 1.0, 1.0, ...",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,245,245,The environment is constructed as a rectangula...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=RectangularArena(le...,"<?xml version=""1.00"" ?>\n<argos-configuration>...",--nroot 3 --nchildroot 3 --n0 0 --nchild0 2 --...,"[20.0, 27.0, 23.0, 21.0, 19.0, 20.0, 20.0, 19....",20.900000,Foraging,TargetLights,RectangularArena,CenteredSwarm,"[16.0, 10.0, 14.0, 15.0, 15.0, 16.0, 13.0, 20....",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,14.600000,"[14.0, 22.0, 15.0, 16.0, 12.0, 14.0, 11.0, 18....",--nroot 3 --nchildroot 2 --n0 0 --nchild0 2 -...,15.900000
246,246,246,The environment is constructed as a rectangula...,"<?xml version=""1.00"" ?>\n<conf

In [5]:
df = original_df
df["type"] = df["parameters"].map(lambda x: type(x.objective_params).__name__)

In [6]:

# Step 2: Group by 'type' and 'dataset_position' and calculate the mean of the average scores
result = df.groupby(['type'])[['llm_dpo_rl_avg_score', 'llm_avg_score']].mean().reset_index()


# Display the results
print(result)

           type  llm_dpo_rl_avg_score  llm_avg_score
0   Aggregation              0.292573       0.307450
1    Connection            -12.690606     -13.242303
2  Distribution            -12.647705     -12.842702
3      Foraging              2.109333       1.944595
